## imports

In [135]:
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from tqdm import tqdm
import json
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
data_pth = Path('../data/')

# Jerry Trial

In [136]:
raw_data = pd.read_csv('../data/beginner.csv')
clean_data = raw_data[(raw_data["INCWAGE"] != 99999999.0) & (raw_data["INCWAGE"] != 0.0)]
clean_data = clean_data.dropna(subset = ['INCWAGE'])
clean_data = clean_data.dropna(how = 'all', axis = 1)
clean_data = clean_data.drop(columns = ['HFLAG', 'ASECFLAG', 'MONTH', 'CPSID', 'SERIAL', 'CPSIDP'])
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11538 entries, 0 to 54733
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   YEAR       11538 non-null  int64  
 1   ASECWTH    11538 non-null  float64
 2   REGION     11538 non-null  int64  
 3   STATEFIP   11538 non-null  int64  
 4   NFAMS      11538 non-null  int64  
 5   PERNUM     11538 non-null  int64  
 6   ASECWT     11538 non-null  float64
 7   AGE        11538 non-null  int64  
 8   SEX        11538 non-null  int64  
 9   RACE       11538 non-null  int64  
 10  MARST      11538 non-null  int64  
 11  BPL        11538 non-null  int64  
 12  EMPSTAT    11538 non-null  int64  
 13  OCC        11538 non-null  int64  
 14  UHRSWORKT  11538 non-null  int64  
 15  WKSTAT     11538 non-null  int64  
 16  EDUC       11538 non-null  int64  
 17  INCWAGE    11538 non-null  float64
 18  OINCWAGE   11538 non-null  float64
dtypes: float64(4), int64(15)
memory usage: 1.8 MB


In [137]:
column = clean_data.columns.copy()
column

Index(['YEAR', 'ASECWTH', 'REGION', 'STATEFIP', 'NFAMS', 'PERNUM', 'ASECWT',
       'AGE', 'SEX', 'RACE', 'MARST', 'BPL', 'EMPSTAT', 'OCC', 'UHRSWORKT',
       'WKSTAT', 'EDUC', 'INCWAGE', 'OINCWAGE'],
      dtype='object')

In [138]:
model_data = clean_data[['EDUC', 'AGE', 'WKSTAT']].astype(str)

In [139]:
model_data = pd.get_dummies(data = model_data)

In [140]:
X = model_data.to_numpy()
y = clean_data['INCWAGE'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

# model = LinearRegression(normalize = True)
# model.fit(X_train, y_train) 
# weight = model.coef_
# bias = model.intercept_
# X_predict = model.predict(X_test)
# model_error = mean_squared_error(y_test, X_predict)
# model.score(X_test, y_test), model_error

# clf = RidgeCV(alphas=[1e-3]).fit(X_train, y_train)
# clf.score(X_test, y_test)

rng = np.random.RandomState(0)
clf = Ridge(alpha = 0.1)
clf.fit(X_train, y_train)
model_err = mean_squared_error(clf.predict(X_test), y_test)
clf.score(X_test, y_test), model_err

(0.2245638814739056, 2664244521.316953)

In [71]:
clean_data.corr()['INCWAGE'].sort_values()

WKSTAT      -0.218311
MARST       -0.192359
OCC         -0.176454
SEX         -0.156439
EMPSTAT     -0.156022
PERNUM      -0.144205
UHRSWORKT   -0.130031
NFAMS       -0.061125
STATEFIP    -0.015878
REGION      -0.012508
RACE        -0.000379
ASECWT       0.003102
BPL          0.016128
ASECWTH      0.017116
YEAR         0.083502
AGE          0.168040
OINCWAGE     0.211331
EDUC         0.326636
INCWAGE      1.000000
Name: INCWAGE, dtype: float64

# End of Jerry Trial

## Constants

In [ ]:
NIU = 99999999
with open(data_pth / 'cpi99_cons.json') as fin:
    cpi99_cons = json.load(fin)

Drop useless columns

In [ ]:
raw_data = pd.read_csv('../data/beginner.csv')
simple_data = raw_data[[
    'YEAR', 'SERIAL', 'REGION', 'NFAMS', 'PERNUM', 
    'AGE', 'SEX', 'EMPSTAT', 'OCC', 'UHRSWORKT', 
    'WKSTAT', 'JOBCERT', 'EDUC', 'INCWAGE', 'OINCWAGE']]
simple_data.to_csv('../data/simple_data.csv', index=False)

In [ ]:
incwage = simple_data['INCWAGE']
print('total reports:', len(incwage))
print('number of 0 wage:', np.count_nonzero(incwage == 0))
print('number of N.I.U wage:', np.count_nonzero(incwage == 99999999))
print('number of ? wage:', np.count_nonzero(incwage == 99999998))
print('number of NaN wage:', np.count_nonzero(incwage.isna()))
print('number of readable wage:', np.count_nonzero((incwage > 0) & (incwage < 99999998)))


In [ ]:
filtered = simple_data[(simple_data['INCWAGE'] >= 0) & (simple_data['INCWAGE'] < NIU)].reset_index(drop=True)

filtered_nonzero = filtered[filtered['INCWAGE'] > 0].reset_index(drop=True)
filtered_nonzero

In [ ]:
import seaborn as sns
corr = filtered_nonzero.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)


In [ ]:
# per individual

In [ ]:
def clean_wage(df):

    df['cpi99'] = df['YEAR'].apply(lambda yr : cpi99_cons[str(yr)])
    df['inc_cpi99'] = df['INCWAGE'] * df['cpi99']

    df.groupby('YEAR').mean()['INCWAGE'].plot(legend=True)
    df.groupby('YEAR').mean()['inc_cpi99'].plot(legend=True)

clean_wage(filtered_nonzero)
clean_wage(filtered)


In [ ]:
def display_categ(df, it):
    # salary = df['inc_salary_cpi99']
    # it = df[it]
    df.groupby(it).mean()['inc_cpi99'].plot(kind="bar", legend=True)
    plt.figure()

indices = ['REGION', 'NFAMS', 'AGE', 'SEX', 'EMPSTAT', 'OCC', 'UHRSWORKT', 'WKSTAT', 'EDUC']

for idx in indices:
    display_categ(filtered, idx)